# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770719044143                   -0.53    9.0         
  2   -2.772137951101       -2.85       -1.29    1.0    107ms
  3   -2.772169823260       -4.50       -2.70    1.0    137ms
  4   -2.772170716032       -6.05       -3.66    2.0    126ms
  5   -2.772170722016       -8.22       -4.04    2.0    129ms
  6   -2.772170722991       -9.01       -5.33    1.0    145ms
  7   -2.772170723014      -10.63       -5.58    2.0    132ms
  8   -2.772170723015      -12.06       -6.35    2.0    142ms
  9   -2.772170723015      -13.78       -6.79    1.0    129ms
 10   -2.772170723015   +  -14.24       -8.07    1.0    133ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770679355104                   -0.53    9.0         
  2   -2.772052507549       -2.86       -1.31    1.0    137ms
  3   -2.7

1.7735578306432633

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770794304900                   -0.52    9.0         
  2   -2.772061539767       -2.90       -1.32    1.0    158ms
  3   -2.772083052267       -4.67       -2.45    1.0    107ms
  4   -2.772083345588       -6.53       -3.15    1.0    115ms
  5   -2.772083417563       -7.14       -4.26    2.0    163ms
  6   -2.772083417704       -9.85       -4.43    1.0    115ms
  7   -2.772083417808       -9.98       -5.56    1.0    137ms
  8   -2.772083417811      -11.57       -5.85    2.0    136ms
  9   -2.772083417811   +  -14.40       -6.20    1.0    132ms
 10   -2.772083417811      -13.55       -7.22    1.0    130ms
 11   -2.772083417811   +  -14.40       -7.57    1.0    133ms
 12   -2.772083417811   +  -15.35       -7.93    1.0    142ms
 13   -2.772083417811   +  -14.65       -9.01    2.0    150ms

Polarizability via ForwardDiff:       1.7725349691456143
Polarizabilit